<a href="https://colab.research.google.com/github/deathreaper17/C-Plus-Plus/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import  numpy as np
import tensorflow as tf
import keras
from keras.layers import Dense, Flatten
from keras.models import Sequential

In [2]:
(X_train_full,y_train_full),(X_test,y_test)= tf.keras.datasets.fashion_mnist.load_data()
X_train_full=X_train_full/ 255.0
X_test=X_test/255.0
X_valid, X_train=X_train_full[:5000], X_train_full[5000:]
y_valid, y_train=y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:

len(X_train)

In [ ]:
def split_dataset(X,y):
  y_5_or_6=(y==5)|(y==6)
  y_A= y[~ y_5_or_6] # ~ is acting as a not gate.
  y_A[y_A >6]
  y_B=(y[y_5_or_6]==6).astype(np.float32)
  return ((X[~y_5_or_6],y_A),(X[y_5_or_6],y_B))

In [ ]:
y= (y_train==5)|(y_train==6)
y

In [ ]:
(X_train_A,y_train_A),(X_train_B,y_train_B )=split_dataset(X_train,y_train)
(X_valid_A,y_valid_A),(X_valid_B,y_valid_B)=split_dataset(X_valid,y_valid)
(X_test_A,y_test_A),(X_test_B,y_test_B)=split_dataset(X_test,y_test)
X_train_B=X_train_B[:200]
y_train_B=y_train_B[:200]

In [ ]:

def spliqt_dataset(y):
  y_5_or_6=(y==5)|(y==6)
  y_A= y[~ y_5_or_6] # ~ is acting as a not gate.
  y_A[y_A >6]
  y_B=(y[y_5_or_6]==6).astype(np.float32)
  return y_A,y_B

In [ ]:
S=np.array([1,2,3,4,5,6,7,8,9,0]) 
ye, yf = spliqt_dataset(S)
yf

In [ ]:
print('\n X_train_A.shape',X_train_A.shape)
print('\n X_train_B.shape',X_train_B.shape)
print('\n y_train_A[:30]',y_train_A[:30])
print('\n y_train_B[:30]',y_train_B[:30])

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
model_A =Sequential()
model_A.add(Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
  model_A.add(Dense(n_hidden,activation="relu"))
model_A.add(Dense(8,activation="softmax"))
model_A.summary()
model_A.compile(loss="sparse_categorical_crossentropy",optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])
# history = model_A.fit(X_train_A,y_train_A,epochs=20,validation_data=(X_valid_A,y_valid_A))

In [ ]:
model_A.summary()
model_A.save("my_model_A.h5")

In [ ]:
model_B =Sequential()
model_B.add(Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
  model_B.add(Dense(n_hidden,activation="relu"))
model_B.add(Dense(1,activation="sigmoid"))
model_B.summary()
model_B.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])
history=model_B.fit(X_train_B,y_train_B,epochs=20,validation_data=(X_valid_B,y_valid_B))

In [ ]:
# model.save('')

**CLONING MODEL A AND USE IT IN MODEL B ON A**

In [ ]:
model_A=keras.models.load_model("my_model_A.h5")
model_B_on_A =keras.models.Sequential(model_A.layers[:1])
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))
model_B_on_A.summary()

**note that model_B_on_A and model_A actually share layers now , so when we train one ,it will update both models.If we want to avoid that,we need to build model_B_on_A on top of a clone of model_A**

In [ ]:
model_A_clone=keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A=keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))

In [ ]:
for layers in model_B_on_A.layers[:-1]:
    layers.trainable=False

In [ ]:
model_B.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])
history=model_B.fit(X_train_B,y_train_B,epochs=4,validation_data=(X_valid_B,y_valid_B))

In [ ]:
for layers in model_B_on_A.layers[:-1]:
    layers.trainable=True

In [ ]:
model_B_on_A.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),metrics=["accuracy"])
history_B_on_A_1=model_B_on_A.fit(X_train_B,y_train_B,epochs=20,validation_data=(X_valid_B,y_valid_B))